In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/moa-comp/stratified_train_kfolds.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_target_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_target_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
test_data = pd.read_csv('/kaggle/input/lish-moa/test_features.csv') 
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv') 

In [3]:
train_data.shape

(23814, 876)

In [4]:
train_data.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [5]:
train_data.sig_id.nunique()

23814

In [6]:
train_data.cp_time.value_counts()

48    8250
72    7792
24    7772
Name: cp_time, dtype: int64

In [7]:
train_data.cp_dose.value_counts()

D1    12147
D2    11667
Name: cp_dose, dtype: int64

In [8]:
train_data.cp_type.value_counts()

trt_cp         21948
ctl_vehicle     1866
Name: cp_type, dtype: int64

In [9]:
train_target_scored.shape

(23814, 207)

In [10]:
train_target_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train_target_scored.sum()[1:].sort_values()

atp-sensitive_potassium_channel_antagonist      1
erbb2_inhibitor                                 1
diuretic                                        6
autotaxin_inhibitor                             6
protein_phosphatase_inhibitor                   6
                                             ... 
serotonin_receptor_antagonist                 404
dopamine_receptor_antagonist                  424
cyclooxygenase_inhibitor                      435
proteasome_inhibitor                          726
nfkb_inhibitor                                832
Length: 206, dtype: object

In [12]:

from sklearn import preprocessing

cat_features = ['cp_type','cp_time','cp_dose']
for feat in cat_features:
    lbl = preprocessing.LabelEncoder() 
    lbl.fit(train_data[feat].astype(str).values)
    test_data.loc[:,feat] = test_data[feat].map(lambda s: '<unknown>' if s not in lbl.classes_ else s)
    lbl.classes_ = np.append(lbl.classes_, '<unknown>')
    train_data.loc[:,feat] = lbl.transform(train_data[feat].astype(str).values)
    test_data.loc[:,feat] = lbl.transform(test_data[feat].astype(str).values)
    

In [13]:
features = [f for f in train_data.columns if f not in ["sig_id"]]
print((features))

['cp_type', 'cp_time', 'cp_dose', 'g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8', 'g-9', 'g-10', 'g-11', 'g-12', 'g-13', 'g-14', 'g-15', 'g-16', 'g-17', 'g-18', 'g-19', 'g-20', 'g-21', 'g-22', 'g-23', 'g-24', 'g-25', 'g-26', 'g-27', 'g-28', 'g-29', 'g-30', 'g-31', 'g-32', 'g-33', 'g-34', 'g-35', 'g-36', 'g-37', 'g-38', 'g-39', 'g-40', 'g-41', 'g-42', 'g-43', 'g-44', 'g-45', 'g-46', 'g-47', 'g-48', 'g-49', 'g-50', 'g-51', 'g-52', 'g-53', 'g-54', 'g-55', 'g-56', 'g-57', 'g-58', 'g-59', 'g-60', 'g-61', 'g-62', 'g-63', 'g-64', 'g-65', 'g-66', 'g-67', 'g-68', 'g-69', 'g-70', 'g-71', 'g-72', 'g-73', 'g-74', 'g-75', 'g-76', 'g-77', 'g-78', 'g-79', 'g-80', 'g-81', 'g-82', 'g-83', 'g-84', 'g-85', 'g-86', 'g-87', 'g-88', 'g-89', 'g-90', 'g-91', 'g-92', 'g-93', 'g-94', 'g-95', 'g-96', 'g-97', 'g-98', 'g-99', 'g-100', 'g-101', 'g-102', 'g-103', 'g-104', 'g-105', 'g-106', 'g-107', 'g-108', 'g-109', 'g-110', 'g-111', 'g-112', 'g-113', 'g-114', 'g-115', 'g-116', 'g-117', 'g-118', 'g-11

In [14]:
num_features = [f for f in train_data.columns if f not in ["sig_id",'cp_type','cp_time','cp_dose']]
print((num_features))

['g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8', 'g-9', 'g-10', 'g-11', 'g-12', 'g-13', 'g-14', 'g-15', 'g-16', 'g-17', 'g-18', 'g-19', 'g-20', 'g-21', 'g-22', 'g-23', 'g-24', 'g-25', 'g-26', 'g-27', 'g-28', 'g-29', 'g-30', 'g-31', 'g-32', 'g-33', 'g-34', 'g-35', 'g-36', 'g-37', 'g-38', 'g-39', 'g-40', 'g-41', 'g-42', 'g-43', 'g-44', 'g-45', 'g-46', 'g-47', 'g-48', 'g-49', 'g-50', 'g-51', 'g-52', 'g-53', 'g-54', 'g-55', 'g-56', 'g-57', 'g-58', 'g-59', 'g-60', 'g-61', 'g-62', 'g-63', 'g-64', 'g-65', 'g-66', 'g-67', 'g-68', 'g-69', 'g-70', 'g-71', 'g-72', 'g-73', 'g-74', 'g-75', 'g-76', 'g-77', 'g-78', 'g-79', 'g-80', 'g-81', 'g-82', 'g-83', 'g-84', 'g-85', 'g-86', 'g-87', 'g-88', 'g-89', 'g-90', 'g-91', 'g-92', 'g-93', 'g-94', 'g-95', 'g-96', 'g-97', 'g-98', 'g-99', 'g-100', 'g-101', 'g-102', 'g-103', 'g-104', 'g-105', 'g-106', 'g-107', 'g-108', 'g-109', 'g-110', 'g-111', 'g-112', 'g-113', 'g-114', 'g-115', 'g-116', 'g-117', 'g-118', 'g-119', 'g-120', 'g-121', 'g-122', 'g

In [15]:

'''for feat in num_features:
    #tr_mean = np.mean(train_data[feat].values,axis=0)
    #tr_std = np.std(train_data[feat].values,axis=0)
    tr_min = train_data[feat].values.min()
    tr_max = train_data[feat].values.max()
    ts_min = test_data[feat].values.min()
    ts_max = test_data[feat].values.max()
    
    train_data.loc[:,feat] = (train_data[feat].values - tr_min) / (tr_max - tr_min)
    test_data.loc[:,feat] = (test_data[feat].values - ts_min) / (ts_max - ts_min)
'''


'for feat in num_features:\n    #tr_mean = np.mean(train_data[feat].values,axis=0)\n    #tr_std = np.std(train_data[feat].values,axis=0)\n    tr_min = train_data[feat].values.min()\n    tr_max = train_data[feat].values.max()\n    ts_min = test_data[feat].values.min()\n    ts_max = test_data[feat].values.max()\n    \n    train_data.loc[:,feat] = (train_data[feat].values - tr_min) / (tr_max - tr_min)\n    test_data.loc[:,feat] = (test_data[feat].values - ts_min) / (ts_max - ts_min)\n'

In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from keras import regularizers
from tensorflow.keras.models import Model
import tensorflow_addons as tfa
from keras import backend as K
from keras import callbacks
from tensorflow import keras
from tensorflow.keras import regularizers

def get_cat_features(inputs,df, cate_features):
    #inputs = []
    outputs = []
    for idx,cat in enumerate(cate_features):
        uniqu_val = int(df[cat].nunique())
        #print(uniqu_val)
        emb_dim = int(min(np.ceil(uniqu_val/2),50))
        inp = inputs[idx]
        out = layers.Embedding(input_dim=uniqu_val+1,output_dim=emb_dim)(inp)
        out = layers.Reshape(target_shape=(emb_dim,))(out)
        #inputs.append(inp)
        outputs.append(out) 
    y = layers.Concatenate()(outputs)  
    return y

def get_num_features(inputs,num_features):
    inputs0 = []
    for idx,cat in enumerate(num_features):
        inp = inputs[idx]
        inputs0.append(inp)
        
    y = layers.Concatenate()(inputs0)    
    return y

def get_inputs(features):
    inputs0 = []
    for num in features:
        inp = layers.Input(shape=(1,))
        inputs0.append(inp)
        
    #y = layers.Concatenate()(inputs0)    
    return inputs0



#tf.keras.utils.plot_model(model, show_shapes = False, show_layer_names= True,
                          #rankdir = 'TB', expand_nested = False, dpi = 96)


/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [17]:
'''
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold



if __name__ == "__main__":
    df = pd.merge(train_data,train_target_scored,on='sig_id')
    df['kfold'] = -1
    total_features = ['sig_id']+features+num_features
    #print(total_features)
    targets = [f for f in df.columns if f not in total_features]
    
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.loc[:,targets].values

    kf = MultilabelStratifiedKFold(n_splits=5)

    for fold, (train, val) in enumerate(kf.split(X=df,y=y)):
        df.loc[val, 'kfold'] = fold

    df.to_csv('./stratified_train_kfolds.csv', index=False)
'''

'\nfrom iterstrat.ml_stratifiers import MultilabelStratifiedKFold\n\n\n\nif __name__ == "__main__":\n    df = pd.merge(train_data,train_target_scored,on=\'sig_id\')\n    df[\'kfold\'] = -1\n    total_features = [\'sig_id\']+features+num_features\n    #print(total_features)\n    targets = [f for f in df.columns if f not in total_features]\n    \n    df = df.sample(frac=1).reset_index(drop=True)\n    y = df.loc[:,targets].values\n\n    kf = MultilabelStratifiedKFold(n_splits=5)\n\n    for fold, (train, val) in enumerate(kf.split(X=df,y=y)):\n        df.loc[val, \'kfold\'] = fold\n\n    df.to_csv(\'./stratified_train_kfolds.csv\', index=False)\n'

In [18]:
def train(fold):
    print(f"fold: {fold}")
    data = pd.read_csv('/kaggle/input/moa-comp/stratified_train_kfolds.csv')
    df_train = data[data.kfold != fold].reset_index(drop=True)
    df_val = data[data.kfold == fold].reset_index(drop=True)

    x_train = [df_train.loc[:,feat].values for feat in features]
    y_train = df_train.drop(['sig_id']+features+['kfold'],axis=1).values

    x_val = [df_val.loc[:,feat].values for feat in features]
    y_val = df_val.drop(['sig_id']+features+['kfold'],axis=1).values

    #print(len(x_train))
    #print(y_train.shape)
    
    inp = get_inputs(features)
    y_cat = get_cat_features(inp,train_data,cat_features)
    y_num = get_num_features(inp,num_features)

    y = layers.Concatenate()([y_cat,y_num])
    y = tfa.layers.WeightNormalization(layers.Dense(units=1655,activation='elu'
                                                    ,kernel_initializer=tf.keras.initializers.GlorotNormal()))(y)

    y = layers.Dropout(0.38)(y)
    y = layers.BatchNormalization()(y)
    
    #y = tfa.layers.WeightNormalization(layers.Dense(units=1655,activation='elu'))(y)
    #y = layers.Dropout(0.38)(y)
    #y = layers.BatchNormalization()(y)
    
    y = tfa.layers.WeightNormalization(layers.Dense(1655,activation='elu',
                                                    kernel_initializer=tf.keras.initializers.GlorotNormal()))(y)

    y = layers.Dropout(0.38)(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dense(206,activation='sigmoid')(y)

    model = Model(inputs=inp,outputs=y)

    #model.summary()
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5,
            verbose=1, mode="min", baseline=None,   restore_best_weights=True)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                      patience=5, min_lr=1e-6, mode='max', verbose=1)
     
    #optimizer=tfa.optimizers.Lookahead(tf.keras.optimizers.Adam(lr=0.00072)
    #learning_rate_fn = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=0.0005)
    sgd=tf.keras.optimizers.Adam(lr=0.00072)                                   
    stocastic_avg_sgd = tfa.optimizers.SWA(sgd)
    #avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath='/kaggle/working', save_weights_only=True,
     #                                               update_weights=True)

    model.compile(loss='binary_crossentropy',
    optimizer=stocastic_avg_sgd)

    model.fit(x_train,y_train,batch_size=261,epochs=21,validation_data=(x_val,y_val))#,callbacks=[avg_callback])
    model.save(f"/kaggle/working/model_{fold}.h5")
    score = model.evaluate(x_val,y_val)
    print(score)
    return score,model

'''
best_param: 
{'hidden_size': 1655, 'dropout': 0.3818704648045326, 'learning_rate': 0.0007188788763769456, 'batch_size': 261}
hidden:512, dropout:0.45,lr:0.008,bs:64/2048
def objective(trial):
    params = {
         #"hidden_size":trial.suggest_int("hidden_size",16,2048),
         #"dropout":trial.suggest_float("dropout",0.1,0.8),
         "learning_rate":trial.suggest_loguniform("learning_rate",1e-6,1e-3)
    }
     
    tot_loss = []
    for fold_ in range(5):
        loss = train(fold_,params)
        tot_loss.append(loss)
    return np.mean(tot_loss) 
'''

'\nbest_param: \n{\'hidden_size\': 1655, \'dropout\': 0.3818704648045326, \'learning_rate\': 0.0007188788763769456, \'batch_size\': 261}\nhidden:512, dropout:0.45,lr:0.008,bs:64/2048\ndef objective(trial):\n    params = {\n         #"hidden_size":trial.suggest_int("hidden_size",16,2048),\n         #"dropout":trial.suggest_float("dropout",0.1,0.8),\n         "learning_rate":trial.suggest_loguniform("learning_rate",1e-6,1e-3)\n    }\n     \n    tot_loss = []\n    for fold_ in range(5):\n        loss = train(fold_,params)\n        tot_loss.append(loss)\n    return np.mean(tot_loss) \n'

In [19]:
'''
from functools import partial
import optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=2)
   
print("best_trial: ")
trial_ = study.best_trial
print("value: {}".format(trial_.value))


print("best_param: ")
best_params = trial_.params
print(best_params)

scores = 0
for j in range(5):
    score = train(fold=j,params=best_params)
    scores += score

print(scores/5)
'''

'\nfrom functools import partial\nimport optuna\n\nstudy = optuna.create_study(direction=\'minimize\')\nstudy.optimize(objective,n_trials=2)\n   \nprint("best_trial: ")\ntrial_ = study.best_trial\nprint("value: {}".format(trial_.value))\n\n\nprint("best_param: ")\nbest_params = trial_.params\nprint(best_params)\n\nscores = 0\nfor j in range(5):\n    score = train(fold=j,params=best_params)\n    scores += score\n\nprint(scores/5)\n'

In [20]:

for i in range(5):
    _,model = train(fold=i)

fold: 0
Epoch 1/21
73/73 [==============================] - 11s 148ms/step - loss: 0.5430 - val_loss: 0.1199
Epoch 2/21
73/73 [==============================] - 6s 86ms/step - loss: 0.0562 - val_loss: 0.0284
Epoch 3/21
73/73 [==============================] - 6s 80ms/step - loss: 0.0242 - val_loss: 0.0200
Epoch 4/21
73/73 [==============================] - 5s 75ms/step - loss: 0.0206 - val_loss: 0.0188
Epoch 5/21
73/73 [==============================] - 6s 80ms/step - loss: 0.0189 - val_loss: 0.0183
Epoch 6/21
73/73 [==============================] - 5s 75ms/step - loss: 0.0180 - val_loss: 0.0173
Epoch 7/21
73/73 [==============================] - 7s 90ms/step - loss: 0.0174 - val_loss: 0.0174
Epoch 8/21
73/73 [==============================] - 6s 80ms/step - loss: 0.0171 - val_loss: 0.0167
Epoch 9/21
73/73 [==============================] - 5s 75ms/step - loss: 0.0165 - val_loss: 0.0166
Epoch 10/21
73/73 [==============================] - 6s 78ms/step - loss: 0.0160 - val_loss: 0.0163

In [21]:
def pred_fct(model,test_data):
    x_test = [test_data.loc[:,feat].values for feat in features]
    preds = model.predict(x_test)
    #print(preds.shape)
    return preds

model_0 = tf.keras.models.load_model("/kaggle/working/model_0.h5")
model_1 = tf.keras.models.load_model("/kaggle/working/model_1.h5")
model_2 = tf.keras.models.load_model("/kaggle/working/model_2.h5")
model_3 = tf.keras.models.load_model("/kaggle/working/model_3.h5")
model_4 = tf.keras.models.load_model("/kaggle/working/model_4.h5")

pred0 = pred_fct(model_0,test_data)
pred1 = pred_fct(model_1,test_data)
pred2 = pred_fct(model_2,test_data)
pred3 = pred_fct(model_3,test_data)
pred4 = pred_fct(model_4,test_data)

avg_pred = (pred0 + pred1 + pred2 + pred3 + pred4)/5 


In [22]:
G_sub = submission
G_labels = [f for f in G_sub.columns if f not in ["sig_id"]]
sub = pd.DataFrame()
sub["sig_id"] = G_sub["sig_id"]
sub[G_labels] = avg_pred
sub.to_csv('./submission.csv', index=False)
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000446,0.002279,0.000986,0.008915,0.005874,0.002047,0.000597,0.003760,0.000152,...,0.000626,0.002785,0.002719,0.000404,0.000220,0.000531,0.000415,0.001584,0.004504,0.000868
1,id_001897cda,0.000058,0.000732,0.000746,0.000476,0.001045,0.001196,0.001756,0.003071,0.002922,...,0.000387,0.000912,0.001954,0.000157,0.004110,0.000238,0.000522,0.000284,0.000933,0.001738
2,id_002429b5b,0.000139,0.000053,0.000637,0.000171,0.010388,0.000092,0.000446,0.000273,0.000086,...,0.000199,0.000122,0.000256,0.000630,0.001898,0.000057,0.003378,0.001118,0.000121,0.000672
3,id_00276f245,0.000121,0.000402,0.000436,0.001620,0.000786,0.003540,0.000356,0.000774,0.000155,...,0.000154,0.000415,0.001609,0.014392,0.007335,0.000247,0.001418,0.000719,0.000260,0.000784
4,id_0027f1083,0.002179,0.002569,0.002600,0.009125,0.017986,0.000918,0.008675,0.001109,0.000772,...,0.001074,0.000589,0.007689,0.003800,0.000548,0.001046,0.003442,0.001781,0.000231,0.001460
